# Procesamiento de los datos

## Exploración de los datos

In [3]:
with open('10pt-300-800K-tri-input.data', 'r') as file:
    data = file.read()

# Separar el archivo en bloques
bloques = data.split('end')

# Imprimir las dos primeras muestras
for i in range(2):
    bloque = bloques[i].strip()  # Eliminar espacios en blanco adicionales al principio o al final
    print(f"Muestra {i + 1}:\n{bloque}\n")

Muestra 1:
begin
 atom    -0.5088619469     0.6465231181    -0.7523580470    Pt    -0.019779   0.00    -0.0061096361    0.0000417696   -0.0090780195 
 atom    -0.1019002647     3.0988826216     3.4420952397    Pt    -0.042923   0.00    -0.0014660338   -0.0016782891    0.0085067604 
 atom     3.8922418528     0.1897968059     3.6961121392    Pt     0.060364   0.00    -0.0012657784   -0.0008649976    0.0045574247 
 atom     2.3536201889    -1.5371028261     8.1018207729    Pt    -0.033898   0.00    -0.0039134251    0.0070078585   -0.0005063838 
 atom     2.8156565049    -5.7685142829     5.7079283639    Pt    -0.023967   0.00    -0.0006972047   -0.0103496587    0.0018396179 
 atom     6.7212719819    -3.8816512388     3.4843918122    Pt    -0.015857   0.00     0.0012600333    0.0011896847   -0.0057454817 
 atom     4.2881427800    -4.3366139264    -0.7696935952    Pt     0.016515   0.00    -0.0010321952    0.0082709918    0.0060865969 
 atom     0.5189413117    -3.1294513934     2.242586

Unicamente los datos correspondientes a la energía total del arreglo (meV), las posiciones(A) y fuerzas de los átomos son de nuestro intéres(meV/A).

## Procesamiento de los datos

### Importación de módulos

In [4]:
import numpy as np
import h5py
import random
import os
import shutil

### Creación de la carpeta auxiliar temp

In [5]:
nombre_carpeta = "temp"

if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)
    os.mkdir(nombre_carpeta)
else:
    os.mkdir(nombre_carpeta)

os.mkdir("datos-procesados")

### Funciones 

In [6]:
#Intercamcio de elemento por su posición en la tabla periodica
def posicion_en_tabla_periodica(elemento_quimico):
    tabla_periodica = {
        "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10,
        "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18,
        "K": 19, "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Ni": 27, "Co": 28,
        "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36,
        "Rb": 37, "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46,
        "Ag": 47, "Cd": 48, "In": 49, "Sn": 50, "Sb": 51, "Te": 52, "I": 53, "Xe": 54,
        "Cs": 55, "Ba": 56, "La": 57, "Hf": 72, "Ta": 73, "W": 74, "Re": 75, "Os": 76, "Ir": 77, "Pt": 78,
        "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83, "Th": 90, "Pa": 91, "U": 92, "Np": 93, "Pu": 94,
        "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99, "Fm": 100
    }

    elemento = elemento_quimico.capitalize()
    if elemento in tabla_periodica:
        return tabla_periodica[elemento]
    else:
        return None

### Exploración de los datos

In [7]:
###################################################
archivo_entrada = "10pt-300-800K-tri-input.data" 
###################################################
    
#Encontrar número de nuestras
num_muestras=0
with open(archivo_entrada, "r") as f_in:
    for linea in f_in:
        linea=linea.split()
        if linea[0]=='begin':
            num_muestras=num_muestras+1

print(f"\nEl archivo tiene {num_muestras} muestras")

#Encontrar número de átomos
i=0
num_atomos=0
with open(archivo_entrada, "r") as f_in:
    for linea in f_in:
        linea=linea.split()
        if linea[0]=='begin':
            i=i+1
        if i==2:
            break
        if linea[0]=='atom':
            num_atomos=num_atomos+1

print(f"\nCada arreglo tiene {num_atomos} atomos")

print(f"\nDando un total de {num_muestras*num_atomos} atomos")


El archivo tiene 11000 muestras

Cada arreglo tiene 10 atomos

Dando un total de 110000 atomos


### Guardar los datos seleccionados en archivos .data

In [8]:
with open(archivo_entrada, "r") as f_in:
    for linea in f_in:
        linea=linea.split()
        if linea[0]=='begin':
            continue
        if linea[0]=='atom':
            n_pos=[linea[1],linea[2],linea[3]]
            z=posicion_en_tabla_periodica(linea[4])
            n_type=[z]
            n_force=[linea[7],linea[8],linea[9]]
            with open("temp/pos.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_pos)
                file.write(line + "\n")
            with open("temp/type.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_type)
                file.write(line + "\n")
            with open("temp/force.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_force)
                file.write(line + "\n")
        if linea[0]=='energy':
            n_energy=[linea[1]]
            with open("temp/energy.data", "a") as file:
                line = " ".join(str(elemento) for elemento in n_energy)
                file.write(line + "\n")
        if linea[0]=='charge':
            continue
        if linea[0]=='end':
            continue

print("\nDatos procesados")

### Cargar datos en numpy arrays

In [ ]:
# Leer los datos del archivo "energy.data"
with open("temp/energy.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
energy = np.array(data)

# Leer los datos del archivo "type.data"
with open("temp/type.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
types = np.array(data)

# Leer los datos del archivo "pos.data"
with open("temp/pos.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
pos = np.array(data)

# Leer los datos del archivo "force.data"
with open("temp/force.data", "r") as f:
    lines = f.readlines()
# Procesar los datos y crear un arreglo NumPy
data = []
for line in lines:
    values = line.strip().split()
    data.append([float(val) for val in values])
# Convertir la lista en un arreglo NumPy
forces = np.array(data)

print("\narray's creados")

### Convertir los datos a las dimensiones correspondientes

In [ ]:
# Acomodo de dimensiones
energy=np.reshape(energy,(num_muestras))
pos=np.reshape(pos,(num_muestras,num_atomos,3))
types=np.reshape(types,(num_muestras,num_atomos))
forces=np.reshape(forces,(num_muestras,num_atomos,3))

print("\nDimensiones acomodadas")

### Separación de train y test

In [ ]:
#Separación train, test

#Generar una selección de indices aleatoria

###################################
seed_value = 42####################
random.seed(seed_value)#############
####################################
num_test=int(0.05*num_muestras)#####
####################################
numeros = list(range(num_muestras))
muestra_aleatoria = random.sample(numeros, num_test)  
muestra_aleatoria.sort()
muestra_aleatoria=np.array(muestra_aleatoria)

energy_test=energy[muestra_aleatoria]
pos_test=pos[muestra_aleatoria]
types_test=types[muestra_aleatoria]
forces_test=forces[muestra_aleatoria]

energy=np.delete(energy,muestra_aleatoria, axis=0)
pos=np.delete(pos,muestra_aleatoria, axis=0)
types=np.delete(types,muestra_aleatoria, axis=0)
forces=np.delete(forces,muestra_aleatoria, axis=0)

### Guardar en archivo .h5 (train)

In [ ]:
#train
########################################################
filename = "datos-procesados/10pt-300-800K-tri-train.h5"##
########################################################

f=h5py.File(filename, "w")

##############################
group_name = "10pt-300-800K-tri"##
##############################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types)
group.create_dataset("pos", data=pos)
group.create_dataset("energy", data=energy)
group.create_dataset("forces", data=forces)

f.close()

### Guardar en archivo .h5 (test)

In [ ]:
#test
############################################################
filename = "datos-procesados/10pt-300-800K-tri-test.h5"##
############################################################

f=h5py.File(filename, "w")

###################################
group_name = "10pt-300-800K-tri-test"##
###################################

group = f.create_group(group_name)

# Guardar datos en el grupo
group.create_dataset("types", data=types_test)
group.create_dataset("pos", data=pos_test)
group.create_dataset("energy", data=energy_test)
group.create_dataset("forces", data=forces_test)

f.close()

print("\nArchivos .h5 creados")

### Eliminación de la carpeta auxiliar

In [ ]:
if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)